# Rangkaian Neural Berulang

Dalam modul sebelumnya, kita telah membincangkan representasi semantik teks yang kaya. Seni bina yang kita gunakan menangkap makna agregat perkataan dalam ayat, tetapi ia tidak mengambil kira **susunan** perkataan, kerana operasi agregasi yang mengikuti embedding menghapuskan maklumat ini daripada teks asal. Oleh kerana model-model ini tidak dapat mewakili susunan perkataan, mereka tidak dapat menyelesaikan tugas yang lebih kompleks atau samar seperti penjanaan teks atau menjawab soalan.

Untuk menangkap makna urutan teks, kita akan menggunakan seni bina rangkaian neural yang dipanggil **rangkaian neural berulang**, atau RNN. Apabila menggunakan RNN, kita menghantar ayat kita melalui rangkaian satu token pada satu masa, dan rangkaian menghasilkan beberapa **keadaan**, yang kemudian kita hantar semula ke rangkaian bersama token seterusnya.

![Imej menunjukkan contoh penjanaan rangkaian neural berulang.](../../../../../translated_images/ms/rnn.27f5c29c53d727b5.webp)

Diberikan urutan input token $X_0,\dots,X_n$, RNN mencipta urutan blok rangkaian neural, dan melatih urutan ini secara hujung ke hujung menggunakan backpropagation. Setiap blok rangkaian mengambil pasangan $(X_i,S_i)$ sebagai input, dan menghasilkan $S_{i+1}$ sebagai hasil. Keadaan akhir $S_n$ atau output $Y_n$ dimasukkan ke dalam pengelas linear untuk menghasilkan keputusan. Semua blok rangkaian berkongsi berat yang sama, dan dilatih secara hujung ke hujung menggunakan satu laluan backpropagation.

> Rajah di atas menunjukkan rangkaian neural berulang dalam bentuk yang tidak digulung (di sebelah kiri), dan dalam representasi berulang yang lebih padat (di sebelah kanan). Adalah penting untuk menyedari bahawa semua Sel RNN mempunyai **berat yang boleh dikongsi**.

Oleh kerana vektor keadaan $S_0,\dots,S_n$ dihantar melalui rangkaian, RNN dapat mempelajari kebergantungan berurutan antara perkataan. Sebagai contoh, apabila perkataan *tidak* muncul di suatu tempat dalam urutan, ia dapat belajar untuk menafikan elemen tertentu dalam vektor keadaan.

Di dalamnya, setiap sel RNN mengandungi dua matriks berat: $W_H$ dan $W_I$, serta bias $b$. Pada setiap langkah RNN, diberikan input $X_i$ dan keadaan input $S_i$, keadaan output dikira sebagai $S_{i+1} = f(W_H\times S_i + W_I\times X_i+b)$, di mana $f$ adalah fungsi pengaktifan (sering $\tanh$).

> Untuk masalah seperti penjanaan teks (yang akan kita bincangkan dalam unit seterusnya) atau terjemahan mesin, kita juga ingin mendapatkan beberapa nilai output pada setiap langkah RNN. Dalam kes ini, terdapat juga matriks lain $W_O$, dan output dikira sebagai $Y_i=f(W_O\times S_i+b_O)$.

Mari kita lihat bagaimana rangkaian neural berulang dapat membantu kita mengklasifikasikan set data berita kita.

> Untuk persekitaran sandbox, kita perlu menjalankan sel berikut untuk memastikan perpustakaan yang diperlukan dipasang, dan data telah dimuatkan terlebih dahulu. Jika anda menjalankan secara tempatan, anda boleh melangkau sel berikut.


In [1]:
import sys
!{sys.executable} -m pip install --quiet tensorflow_datasets==4.4.0
!cd ~ && wget -q -O - https://mslearntensorflowlp.blob.core.windows.net/data/tfds-ag-news.tgz | tar xz

In [2]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

# We are going to be training pretty large models. In order not to face errors, we need
# to set tensorflow option to grow GPU memory allocation when required
physical_devices = tf.config.list_physical_devices('GPU') 
if len(physical_devices)>0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

ds_train, ds_test = tfds.load('ag_news_subset').values()

Apabila melatih model berskala besar, peruntukan memori GPU mungkin menjadi masalah. Kita juga mungkin perlu mencuba saiz minibatch yang berbeza, supaya data dapat dimuatkan ke dalam memori GPU kita, namun latihan tetap cukup pantas. Jika anda menjalankan kod ini pada mesin GPU anda sendiri, anda boleh mencuba melaraskan saiz minibatch untuk mempercepatkan proses latihan.

> **Nota**: Versi tertentu pemacu NVidia diketahui tidak melepaskan memori selepas melatih model. Kami menjalankan beberapa contoh dalam buku nota ini, dan ia mungkin menyebabkan memori habis dalam sesetengah konfigurasi, terutamanya jika anda melakukan eksperimen anda sendiri dalam buku nota yang sama. Jika anda menghadapi ralat pelik semasa memulakan latihan model, anda mungkin mahu memulakan semula kernel buku nota.


In [3]:
batch_size = 16
embed_size = 64

## Pengelas RNN Ringkas

Dalam kes RNN ringkas, setiap unit berulang adalah rangkaian linear mudah, yang menerima vektor input dan vektor keadaan, dan menghasilkan vektor keadaan baharu. Dalam Keras, ini boleh diwakili oleh lapisan `SimpleRNN`.

Walaupun kita boleh menghantar token yang dikodkan satu-panas terus ke lapisan RNN, ini bukan idea yang baik kerana dimensi mereka yang tinggi. Oleh itu, kita akan menggunakan lapisan embedding untuk mengurangkan dimensi vektor perkataan, diikuti oleh lapisan RNN, dan akhirnya pengelas `Dense`.

> **Nota**: Dalam kes di mana dimensi tidak begitu tinggi, contohnya apabila menggunakan tokenisasi peringkat aksara, mungkin masuk akal untuk menghantar token yang dikodkan satu-panas terus ke sel RNN.


In [4]:
vocab_size = 20000

vectorizer = keras.layers.experimental.preprocessing.TextVectorization(
    max_tokens=vocab_size,
    input_shape=(1,))

model = keras.models.Sequential([
    vectorizer,
    keras.layers.Embedding(vocab_size, embed_size),
    keras.layers.SimpleRNN(16),
    keras.layers.Dense(4,activation='softmax')
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization (TextVect (None, None)              0         
_________________________________________________________________
embedding (Embedding)        (None, None, 64)          1280000   
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 16)                1296      
_________________________________________________________________
dense (Dense)                (None, 4)                 68        
Total params: 1,281,364
Trainable params: 1,281,364
Non-trainable params: 0
_________________________________________________________________


> **Nota:** Kami menggunakan lapisan embedding yang tidak dilatih di sini untuk kesederhanaan, tetapi untuk hasil yang lebih baik, kami boleh menggunakan lapisan embedding yang telah dilatih menggunakan Word2Vec, seperti yang diterangkan dalam unit sebelumnya. Ia akan menjadi latihan yang baik untuk anda menyesuaikan kod ini agar berfungsi dengan embedding yang telah dilatih.

Sekarang mari kita latih RNN kita. Secara umum, RNN agak sukar untuk dilatih, kerana apabila sel RNN diurai sepanjang panjang urutan, bilangan lapisan yang terlibat dalam backpropagation menjadi sangat besar. Oleh itu, kita perlu memilih kadar pembelajaran yang lebih kecil, dan melatih rangkaian pada dataset yang lebih besar untuk menghasilkan keputusan yang baik. Ini boleh mengambil masa yang agak lama, jadi penggunaan GPU adalah lebih disarankan.

Untuk mempercepatkan proses, kita hanya akan melatih model RNN pada tajuk berita, dengan mengabaikan deskripsi. Anda boleh mencuba melatih dengan deskripsi dan melihat sama ada anda boleh membuat model tersebut dilatih.


In [5]:
def extract_title(x):
    return x['title']

def tupelize_title(x):
    return (extract_title(x),x['label'])

print('Training vectorizer')
vectorizer.adapt(ds_train.take(2000).map(extract_title))

Training vectorizer


In [6]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize_title).batch(batch_size),validation_data=ds_test.map(tupelize_title).batch(batch_size))

7500/7500 [==============================] - 82s 11ms/step - loss: 0.6629 - acc: 0.7623 - val_loss: 0.5559 - val_acc: 0.7995


> **Nota** bahawa ketepatan mungkin lebih rendah di sini, kerana kami hanya melatih pada tajuk berita.


## Meninjau semula urutan pembolehubah

Ingat bahawa lapisan `TextVectorization` secara automatik akan menambah pad pada urutan panjang berubah dalam satu minibatch dengan token pad. Ternyata token tersebut juga terlibat dalam latihan, dan ini boleh menyulitkan penumpuan model.

Terdapat beberapa pendekatan yang boleh kita ambil untuk mengurangkan jumlah padding. Salah satunya adalah dengan menyusun semula dataset mengikut panjang urutan dan mengelompokkan semua urutan mengikut saiz. Ini boleh dilakukan menggunakan fungsi `tf.data.experimental.bucket_by_sequence_length` (lihat [dokumentasi](https://www.tensorflow.org/api_docs/python/tf/data/experimental/bucket_by_sequence_length)).

Pendekatan lain adalah dengan menggunakan **masking**. Dalam Keras, beberapa lapisan menyokong input tambahan yang menunjukkan token mana yang perlu diambil kira semasa latihan. Untuk memasukkan masking ke dalam model kita, kita boleh menambah lapisan `Masking` yang berasingan ([docs](https://keras.io/api/layers/core_layers/masking/)), atau kita boleh menetapkan parameter `mask_zero=True` pada lapisan `Embedding` kita.

> **Note**: Latihan ini akan mengambil masa sekitar 5 minit untuk melengkapkan satu epoch pada keseluruhan dataset. Anda boleh menghentikan latihan pada bila-bila masa jika anda kehilangan kesabaran. Apa yang juga boleh dilakukan adalah menghadkan jumlah data yang digunakan untuk latihan, dengan menambah klausa `.take(...)` selepas dataset `ds_train` dan `ds_test`.


In [7]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

model = keras.models.Sequential([
    vectorizer,
    keras.layers.Embedding(vocab_size,embed_size,mask_zero=True),
    keras.layers.SimpleRNN(16),
    keras.layers.Dense(4,activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

7500/7500 [==============================] - 371s 49ms/step - loss: 0.5401 - acc: 0.8079 - val_loss: 0.3780 - val_acc: 0.8822


Sekarang kita menggunakan penutupan, kita boleh melatih model pada keseluruhan dataset tajuk dan deskripsi.

> **Note**: Adakah anda perasan bahawa kita telah menggunakan vectorizer yang dilatih pada tajuk berita, dan bukan keseluruhan isi artikel? Kemungkinan, ini boleh menyebabkan beberapa token diabaikan, jadi adalah lebih baik untuk melatih semula vectorizer. Walau bagaimanapun, kesannya mungkin sangat kecil, jadi kita akan kekal menggunakan vectorizer yang telah dilatih sebelumnya demi kesederhanaan.


## LSTM: Memori jangka panjang pendek

Salah satu masalah utama RNN ialah **kecerunan yang hilang**. RNN boleh menjadi agak panjang, dan mungkin sukar untuk menyebarkan kecerunan kembali ke lapisan pertama rangkaian semasa backpropagation. Apabila ini berlaku, rangkaian tidak dapat mempelajari hubungan antara token yang jauh. Salah satu cara untuk mengelakkan masalah ini ialah dengan memperkenalkan **pengurusan keadaan secara eksplisit** menggunakan **pintu kawalan**. Dua seni bina yang paling biasa yang memperkenalkan pintu kawalan ialah **memori jangka panjang pendek** (LSTM) dan **unit relay berpintu** (GRU). Kita akan membincangkan LSTM di sini.

![Imej menunjukkan contoh sel memori jangka panjang pendek](../../../../../lessons/5-NLP/16-RNN/images/long-short-term-memory-cell.svg)

Rangkaian LSTM disusun dengan cara yang serupa dengan RNN, tetapi terdapat dua keadaan yang dihantar dari lapisan ke lapisan: keadaan sebenar $c$, dan vektor tersembunyi $h$. Pada setiap unit, vektor tersembunyi $h_{t-1}$ digabungkan dengan input $x_t$, dan bersama-sama mereka mengawal apa yang berlaku kepada keadaan $c_t$ dan output $h_{t}$ melalui **pintu kawalan**. Setiap pintu kawalan mempunyai pengaktifan sigmoid (output dalam julat $[0,1]$), yang boleh dianggap sebagai topeng bitwise apabila didarabkan dengan vektor keadaan. LSTM mempunyai pintu kawalan berikut (dari kiri ke kanan pada gambar di atas):
* **pintu lupa** yang menentukan komponen mana dalam vektor $c_{t-1}$ yang perlu kita lupakan, dan mana yang perlu diteruskan.
* **pintu input** yang menentukan berapa banyak maklumat daripada vektor input dan vektor tersembunyi sebelumnya yang harus dimasukkan ke dalam vektor keadaan.
* **pintu output** yang mengambil vektor keadaan baru dan memutuskan komponen mana yang akan digunakan untuk menghasilkan vektor tersembunyi baru $h_t$.

Komponen keadaan $c$ boleh dianggap sebagai bendera yang boleh dihidupkan dan dimatikan. Sebagai contoh, apabila kita menemui nama *Alice* dalam urutan, kita menganggap ia merujuk kepada seorang wanita, dan menaikkan bendera dalam keadaan yang mengatakan kita mempunyai kata nama perempuan dalam ayat. Apabila kita seterusnya menemui perkataan *dan Tom*, kita akan menaikkan bendera yang mengatakan kita mempunyai kata nama jamak. Oleh itu, dengan memanipulasi keadaan, kita boleh menjejaki sifat tatabahasa ayat.

> **Note**: Berikut adalah sumber yang hebat untuk memahami struktur dalaman LSTM: [Understanding LSTM Networks](https://colah.github.io/posts/2015-08-Understanding-LSTMs/) oleh Christopher Olah.

Walaupun struktur dalaman sel LSTM mungkin kelihatan kompleks, Keras menyembunyikan pelaksanaannya di dalam lapisan `LSTM`, jadi satu-satunya perkara yang perlu kita lakukan dalam contoh di atas ialah menggantikan lapisan berulang:


In [8]:
model = keras.models.Sequential([
    vectorizer,
    keras.layers.Embedding(vocab_size, embed_size),
    keras.layers.LSTM(8),
    keras.layers.Dense(4,activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(8),validation_data=ds_test.map(tupelize).batch(8))

15000/15000 [==============================] - 188s 13ms/step - loss: 0.5692 - acc: 0.7916 - val_loss: 0.3441 - val_acc: 0.8870


## RNN Dwihala dan Berlapis

Dalam contoh kita sebelum ini, rangkaian berulang beroperasi dari permulaan urutan hingga ke penghujung. Ini terasa semula jadi bagi kita kerana ia mengikuti arah yang sama seperti kita membaca atau mendengar ucapan. Walau bagaimanapun, untuk senario yang memerlukan akses rawak kepada urutan input, lebih masuk akal untuk menjalankan pengiraan berulang dalam kedua-dua arah. RNN yang membenarkan pengiraan dalam kedua-dua arah dipanggil **RNN dwihala**, dan ia boleh dicipta dengan membungkus lapisan berulang dengan lapisan khas `Bidirectional`.

> **Note**: Lapisan `Bidirectional` membuat dua salinan lapisan di dalamnya, dan menetapkan sifat `go_backwards` pada salah satu salinan tersebut kepada `True`, menjadikannya bergerak dalam arah bertentangan sepanjang urutan.

Rangkaian berulang, sama ada sehala atau dwihala, menangkap corak dalam urutan, dan menyimpannya ke dalam vektor keadaan atau mengembalikannya sebagai output. Seperti rangkaian konvolusi, kita boleh membina lapisan berulang lain selepas yang pertama untuk menangkap corak tahap lebih tinggi, yang dibina daripada corak tahap lebih rendah yang diekstrak oleh lapisan pertama. Ini membawa kita kepada konsep **RNN berlapis**, yang terdiri daripada dua atau lebih rangkaian berulang, di mana output lapisan sebelumnya dihantar ke lapisan seterusnya sebagai input.

![Imej menunjukkan RNN LSTM berlapis panjang-pendek](../../../../../translated_images/ms/multi-layer-lstm.dd975e29bb2a59fe.webp)

*Gambar daripada [post yang hebat ini](https://towardsdatascience.com/from-a-lstm-cell-to-a-multilayer-lstm-network-with-pytorch-2899eb5696f3) oleh Fernando López.*

Keras memudahkan pembinaan rangkaian ini, kerana anda hanya perlu menambah lebih banyak lapisan berulang ke model. Untuk semua lapisan kecuali yang terakhir, kita perlu menentukan parameter `return_sequences=True`, kerana kita memerlukan lapisan untuk mengembalikan semua keadaan perantaraan, dan bukan hanya keadaan akhir pengiraan berulang.

Mari kita bina LSTM dwihala dua lapisan untuk masalah klasifikasi kita.

> **Note** kod ini sekali lagi mengambil masa yang agak lama untuk diselesaikan, tetapi ia memberikan ketepatan tertinggi yang pernah kita lihat setakat ini. Jadi mungkin ia berbaloi untuk menunggu dan melihat hasilnya.


In [9]:
model = keras.models.Sequential([
    vectorizer,
    keras.layers.Embedding(vocab_size, 128, mask_zero=True),
    keras.layers.Bidirectional(keras.layers.LSTM(64,return_sequences=True)),
    keras.layers.Bidirectional(keras.layers.LSTM(64)),    
    keras.layers.Dense(4,activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(batch_size),
          validation_data=ds_test.map(tupelize).batch(batch_size))

5045/7500 [===================>..........] - ETA: 2:33 - loss: 0.3709 - acc: 0.8706

## RNN untuk tugas lain

Sehingga kini, kita telah memberi tumpuan kepada penggunaan RNN untuk mengklasifikasikan urutan teks. Namun, ia boleh menangani banyak lagi tugas, seperti penjanaan teks dan terjemahan mesin — kita akan membincangkan tugas-tugas tersebut dalam unit seterusnya.



---

**Penafian**:  
Dokumen ini telah diterjemahkan menggunakan perkhidmatan terjemahan AI [Co-op Translator](https://github.com/Azure/co-op-translator). Walaupun kami berusaha untuk memastikan ketepatan, sila ambil perhatian bahawa terjemahan automatik mungkin mengandungi kesilapan atau ketidaktepatan. Dokumen asal dalam bahasa asalnya harus dianggap sebagai sumber yang berwibawa. Untuk maklumat penting, terjemahan manusia profesional adalah disyorkan. Kami tidak bertanggungjawab atas sebarang salah faham atau salah tafsir yang timbul daripada penggunaan terjemahan ini.
